In [11]:
%matplotlib inline

from matplotlib import pyplot as pl

In [15]:
from __future__ import unicode_literals

from keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional, Flatten, Conv1D, MaxPooling1D, Masking, Conv2D, MaxPool2D
from keras.initializers import RandomUniform
import plac
import collections
import random

from keras.utils import plot_model,Progbar

import pathlib
import cytoolz
import numpy
from keras.models import Sequential, Model, model_from_json
from keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional, Flatten, Conv1D, MaxPool1D, Masking, Conv2D, MaxPool2D
from keras.layers import TimeDistributed
from keras.optimizers import Adam
#import cPickle as pickle
import pickle
from sklearn.preprocessing import OneHotEncoder

from collections import Counter, defaultdict

import spacy

from __future__ import print_function
from sklearn.model_selection import train_test_split
import os
import tqdm
import string
import pandas as pd
import numpy as np
from random import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Embedding, Input, RepeatVector
from keras.utils import np_utils
from keras.preprocessing import text, sequence
import numpy as np
import io
import keras

import spacy

Using TensorFlow backend.


In [52]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15464084149033071271
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 291241984
locality {
  bus_id: 1
}
incarnation: 7380962248303043918
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 14381572499614755162
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12589508954143041885
physical_device_desc: "device: XLA_CPU device"
]


In [82]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/gpu:0']

In [57]:

#Method to compute the accruarcy. Call predict_labels to get the labels for the dataset
def compute_f1(predictions, correct, idx2Label): 
    label_pred = []    
    for sentence in predictions:
        label_pred.append([idx2Label[element] for element in sentence])
        
    label_correct = []    
    for sentence in correct:
        label_correct.append([idx2Label[element] for element in sentence])
            
    
    #print label_pred
    #print label_correct
    
    prec = compute_precision(label_pred, label_correct)
    rec = compute_precision(label_correct, label_pred)
    
    f1 = 0
    if (rec+prec) > 0:
        f1 = 2.0 * prec * rec / (prec + rec);
        
    return prec, rec, f1

def compute_precision(guessed_sentences, correct_sentences):
    assert(len(guessed_sentences) == len(correct_sentences))
    correctCount = 0
    count = 0
    
    
    for sentenceIdx in range(len(guessed_sentences)):
        guessed = guessed_sentences[sentenceIdx]
        correct = correct_sentences[sentenceIdx]
        assert(len(guessed) == len(correct))
        idx = 0
        while idx < len(guessed):
            if guessed[idx][0] == 'B': #A new chunk starts
                count += 1
                
                if guessed[idx] == correct[idx]:
                    idx += 1
                    correctlyFound = True
                    
                    while idx < len(guessed) and guessed[idx][0] == 'I': #Scan until it no longer starts with I
                        if guessed[idx] != correct[idx]:
                            correctlyFound = False
                        
                        idx += 1
                    
                    if idx < len(guessed):
                        if correct[idx][0] == 'I': #The chunk in correct was longer
                            correctlyFound = False
                        
                    
                    if correctlyFound:
                        correctCount += 1
                else:
                    idx += 1
            else:  
                idx += 1
    
    precision = 0
    if count > 0:
        print(correctCount)
        precision = float(correctCount) / count
        
    return precision

In [58]:
nlp = spacy.load('en_core_web_lg')

In [59]:
max_length_sent=120
def get_ner_sentences(filex):
    sent_count = 0
    with io.open(filex, encoding='utf-8') as fl:
        for line in fl:
            line = line.strip()
            toks = line.split()
            #sentence break
            if len(toks)<1:
                if len(sent)>0:
                    sent_count+=1
                    if sent_count%1000==0:
                        print(sent_count, sent)
                    doc = nlp(sent)
                    sent= ""
                    yield doc
                    
            #sentence start
            elif toks[0]==u'-DOCSTART-':
                sent=""
            else:
                sent+=toks[0]+' '    
                
#train_docs = get_ner_sentences('eng.train')
#train_docs = list(train_docs)

In [60]:
def get_ner_bio(filex):
    sent_count = 0
    sent = []
    with io.open(filex, encoding='utf-8') as fl:
        for line in fl:
            line = line.strip()
            toks = line.split()
            #sentence break
            if len(toks)<1:
                if len(sent)>0:
                    sent_count+=1
                    if sent_count%100==0:
                        print(sent_count, sent)
                    res = sent
                    sent = []                   
                    yield res
                    
            #sentence start
            elif toks[0]==u'-DOCSTART-':
                sent=[]
            else:
                sent.append(toks[3]+' ')

In [61]:
filex='eng.train'
with io.open(filex, encoding='utf-8') as fl:
    text1=fl.read()
filex='eng.testb'
with io.open(filex, encoding='utf-8') as fl:
    text2=fl.read()

text = text1+text2
chars = sorted(list(set(text)))

print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars)) 

total chars: 88


In [62]:
def pad_hotencode_labels(train_labels, BIO_LABELS, max_length=150, bio=False): 
    if bio:
        LABELS = BIO_LABELS
        ent2id = dict(zip(LABELS, np.arange(len(LABELS))))
        train_encids = [[ent2id[i] for i in ents_list] for ents_list in train_labels ]
    else:
        LABELS = list(set([i.split('-')[-1] for i in BIO_LABELS]))
        ent2id = dict(zip(LABELS, np.arange(len(LABELS))))
        #train_encids = [[ent2id[i] for i in ents_list] for ents_list in train_labels ]
        train_encids = [[ent2id[i.split('-')[-1]] for i in ents_list] for ents_list in train_labels ]
            
    encoder = OneHotEncoder()
    encoder.fit(np.arange(len(LABELS)).reshape(len(LABELS),1))

    Xs = np.zeros((len(train_labels), max_length), dtype='int32')

    for i in range(len(train_encids)):
        sent=train_encids[i]
        for idx, enc in enumerate(sent):
            #print(idx)
            try:
                Xs[i, idx]=enc
            except:
                print(sent)
                
    tr = Xs.reshape(Xs.shape[0], Xs.shape[1], 1)
    tr_he = np.array([encoder.transform(tr[i,:,:]).toarray() for i in range(tr.shape[0])])
    return tr_he, ent2id

In [63]:
def get_embeddings(vocab):
    max_rank = max(lex.rank+1 for lex in vocab if lex.has_vector)
    vectors = numpy.ndarray((max_rank+1, vocab.vectors_length), dtype='float32')
    for lex in vocab:
        if lex.has_vector:
            vectors[lex.rank + 1] = lex.vector
    return vectors

In [64]:
def get_char_emb_features(docs, max_sentence_length=150, max_word_length=52):
    #docs = list(docs)
    Xs = numpy.zeros((len(docs), max_sentence_length, max_word_length), dtype='int32')
    for i, sentence in enumerate(docs):
        j = 0
        #print(len(sentence))
        for token in sentence:
            for char_id, char in enumerate(token.text):
                if char_id >= max_word_length:
                    break
                Xs[i,j,char_id] = char_indices[char]

            j += 1
    return Xs

# 0 = PADDING
# 1 = missing vector
def get_word_emb_features(docs, max_length):
    #docs = list(docs)
    Xs = numpy.zeros((len(docs), max_length), dtype='int32')
    for i, sentence in enumerate(docs):
        j = 0
        for token in sentence:
            #TODO: els add random vector with id=1 vector = np.random.uniform(-0.25, 0.25, len(split)-1)
            if token.has_vector and not token.is_punct and not token.is_space:
                Xs[i, j] = token.rank + 1
                j += 1
                if j >= max_length:
                    break
    return Xs    

In [31]:
import wget
urls = {"https://github.com/Hironsan/anago/blob/master/data/conll2003/en/raw/train.txt",
        "https://github.com/Hironsan/anago/blob/master/data/conll2003/en/raw/test.txt",
        "https://github.com/Hironsan/anago/blob/master/data/conll2003/en/raw/valid.txt"
       }

for url in urls:
    filename = wget.download(url)
    print(filename)

test.txt
train.txt
valid.txt


In [98]:
#file_str = 'eng.testa'
def create_matrices(file_str):
    max_sent_length = 150  
    doc_labels = list(get_ner_bio(file_str))
    BIO_LABELS = list(set([i for sub_list in doc_labels for i in sub_list]))
    #print(BIO_LABELS)
    LABELS = list(set([i.split('-')[-1] for i in BIO_LABELS]))
    doc_labels, ent2id = pad_hotencode_labels(doc_labels, BIO_LABELS, max_length=max_sent_length, bio=True)
    
    #Memory is not an issue
    docs = list(get_ner_sentences(file_str))
    word_X = get_word_emb_features(docs, max_sent_length)
    char_X = get_char_emb_features(docs)
    
    doc_labels = np.argmax(doc_labels, axis=2).reshape((len(doc_labels), max_sent_length, 1))
    return word_X, char_X, doc_labels, LABELS, ent2id

trainF = "train.txt"
devF = "test.txt"
testF = "valid.txt" #Be careful RAW Dataset

dev_word, dev_char, dev_labels, LABELS, ent2id = create_matrices(devF)
train_word, train_char, train_labels, LABELS, ent2id = create_matrices(trainF)

100 ['B-PER ', 'I-PER ', 'O ', 'B-PER ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ']
200 ['O ', 'O ', 'O ', 'B-LOC ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'B-LOC ', 'O ', 'B-LOC ', 'O ', 'B-LOC ', 'O ', 'B-LOC ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ']
300 ['B-LOC ', 'O ']
400 ['B-ORG ', 'O ', 'B-LOC ']
500 ['O ', 'O ', 'O ', 'O ', 'B-ORG ']
600 ['O ', 'O ', 'O ', 'O ', 'O ', 'O ']
700 ['O ', 'O ', 'B-PER ', 'I-PER ', 'O ', 'B-ORG ', 'O ', 'O ', 'B-PER ', 'I-PER ', 'I-PER ', 'O ', 'B-ORG ', 'O ', 'O ', 'B-PER ', 'I-PER ', 'O ', 'B-ORG ', 'O ', 'O ', 'B-PER ', 'I-PER ', 'O ', 'B-ORG ', 'O ', 'B-LOC ', 'O ', 'O ', 'B-PER ', 'I-PER ', 'O ', 'B-ORG ', 'I-ORG ', 'O ', 'B-LOC ', 'O ', 'O ']
800 ['O ', 'O ', 'O ', 'O ', 'O ', 'B-MISC ', 'O ', 'B-ORG ', 'I-ORG ', 'I-ORG ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'B-PER ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'B-PER ', 'O ', 'O ', 'O ', 'O ', 'O ']
900 ['B-PER ', 'O ', 'O ', 'B-PER ', 'I-PER ', 'O

11100 ['O ', 'O ', 'O ', 'O ', 'B-LOC ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ']
11200 ['O ', 'B-ORG ', 'I-ORG ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ']
11300 ['O ', 'B-MISC ', 'B-ORG ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'B-ORG ', 'I-ORG ', 'I-ORG ', 'I-ORG ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ']
11400 ['O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ']
11500 ['O ', 'O ', 'O ', 'O ', 'B-LOC ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'B-MISC ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'B-LOC ', 'O ', 'O ']
11600 ['B-LOC ', 'O ', 'B-LOC ', 'O ']
11700 ['O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'B-MISC ', 'O ', 'O ', 'O ', 'B-MISC ', 'O ', 'O ', 'B-LOC ', 'I-LOC ', 'O ', 'B-LOC ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ']
11800 ['O ', 'O ', 'O ', 'O ', 'O ', 'B-ORG ', 'I-ORG ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'B-O

In [100]:
train_labels.shape, train_word.shape, train_char.shape

((14041, 150, 1), (14041, 150), (14041, 150, 52))

In [67]:
#save train_labels, train_word, train_char
np.save('train_labels.npy',train_labels)
np.save('train_word.npy',train_word)
np.save('train_char.npy',train_char)

In [101]:
id2ent = dict((v, k) for k, v in ent2id.items())
id2ent

{0: 'B-ORG ',
 1: 'B-PER ',
 2: 'I-MISC ',
 3: 'I-PER ',
 4: 'I-ORG ',
 5: 'B-LOC ',
 6: 'I-LOC ',
 7: 'O ',
 8: 'B-MISC '}

In [102]:
len(id2ent)

9

In [103]:
id2ent[1]

'B-PER '

In [104]:
ent2id

{'B-LOC ': 5,
 'B-MISC ': 8,
 'B-ORG ': 0,
 'B-PER ': 1,
 'I-LOC ': 6,
 'I-MISC ': 2,
 'I-ORG ': 4,
 'I-PER ': 3,
 'O ': 7}

In [72]:
def get_sent_batches(train_word, train_char, train_labels):
    wx = pd.DataFrame(train_word)
    wx['sent_len'] = (wx>0).sum(axis=1)
#     sent_lens = (train_word>0).sum(axis=1)
    for sent_len, group in wx.groupby('sent_len'):
        if sent_len>0:
            batch_word = group.drop('sent_len',axis=1).values[:, 0:sent_len]
            batch_char = train_char[group.index][:, 0:sent_len,:]
            batch_labels = train_labels[group.index][:, 0:sent_len]
            #model.train_on_batch([batch_word, batch_char], batch_labels)
            #print(sent_len, batch_word.shape, batch_char.shape, batch_labels.shape)
            yield (batch_word, batch_char, batch_labels)
            
train_batches = (get_sent_batches(train_word, train_char, train_labels))
dev_batches = (get_sent_batches(dev_word, dev_char, dev_labels))

In [73]:
len(list(train_batches)),len(list(train_labels))

(58, 14041)

In [74]:
#Needed is train_steps
batch_size=20
train_steps = int((len(train_labels) - 1) / batch_size) + 1
dev_steps =   int((len(dev_labels) - 1) / batch_size) + 1
print (train_steps,dev_steps)

703 173


In [168]:
#insert the ChainCRF here
from __future__ import absolute_import
"""
Adapted from: https://github.com/phipleg/keras/blob/crf/keras/layers/crf.py
"""
from keras import backend as K
from keras import initializers, regularizers, constraints
from keras.engine import Layer, InputSpec

def path_energy(y, x, U, b_start=None, b_end=None, mask=None):
    x = add_boundary_energy(x, b_start, b_end, mask)
    return path_energy0(y, x, U, mask)

def path_energy0(y, x, U, mask=None):
    n_classes = K.shape(x)[2]
    y_one_hot = K.one_hot(y, n_classes)

    # Tag path energy
    energy = K.sum(x * y_one_hot, 2)
    energy = K.sum(energy, 1)

    # Transition energy
    y_t = y[:, :-1]
    y_tp1 = y[:, 1:]
    U_flat = K.reshape(U, [-1])
    # Convert 2-dim indices (y_t, y_tp1) of U to 1-dim indices of U_flat:
    flat_indices = y_t * n_classes + y_tp1
    U_y_t_tp1 = K.gather(U_flat, flat_indices)

    if mask is not None:
        mask = K.cast(mask, K.floatx())
        y_t_mask = mask[:, :-1]
        y_tp1_mask = mask[:, 1:]
        U_y_t_tp1 *= y_t_mask * y_tp1_mask

    energy += K.sum(U_y_t_tp1, axis=1)

    return energy


def sparse_chain_crf_loss(y, x, U, b_start=None, b_end=None, mask=None):
    x = add_boundary_energy(x, b_start, b_end, mask)
    energy = path_energy0(y, x, U, mask)
    energy -= free_energy0(x, U, mask)
    return K.expand_dims(-energy, -1)

def chain_crf_loss(y, x, U, b_start=None, b_end=None, mask=None):
    y_sparse = K.argmax(y, -1)
    y_sparse = K.cast(y_sparse, 'int32')
    return sparse_chain_crf_loss(y_sparse, x, U, b_start, b_end, mask)


def add_boundary_energy(x, b_start=None, b_end=None, mask=None):
    if mask is None:
        if b_start is not None:
            x = K.concatenate([x[:, :1, :] + b_start, x[:, 1:, :]], axis=1)
        if b_end is not None:
            x = K.concatenate([x[:, :-1, :], x[:, -1:, :] + b_end], axis=1)
    else:
        mask = K.cast(mask, K.floatx())
        mask = K.expand_dims(mask, 2)
        x *= mask
        if b_start is not None:
            mask_r = K.concatenate([K.zeros_like(mask[:, :1]), mask[:, :-1]], axis=1)
            start_mask = K.cast(K.greater(mask, mask_r), K.floatx())
            x = x + start_mask * b_start
        if b_end is not None:
            mask_l = K.concatenate([mask[:, 1:], K.zeros_like(mask[:, -1:])], axis=1)
            end_mask = K.cast(K.greater(mask, mask_l), K.floatx())
            x = x + end_mask * b_end
    return x


def viterbi_decode(x, U, b_start=None, b_end=None, mask=None):
    x = add_boundary_energy(x, b_start, b_end, mask)

    alpha_0 = x[:, 0, :]
    gamma_0 = K.zeros_like(alpha_0)
    initial_states = [gamma_0, alpha_0]
    _, gamma = _forward(x,
                        lambda B: [K.cast(K.argmax(B, axis=1), K.floatx()), K.max(B, axis=1)],
                        initial_states,
                        U,
                        mask)
    y = _backward(gamma, mask)
    return y


def free_energy(x, U, b_start=None, b_end=None, mask=None):
    """Computes efficiently the sum of all path energies for input x, when
    runs over all possible tag sequences."""
    x = add_boundary_energy(x, b_start, b_end, mask)
    return free_energy0(x, U, mask)


def free_energy0(x, U, mask=None):
    """Free energy without boundary potential handling."""
    initial_states = [x[:, 0, :]]
    last_alpha, _ = _forward(x,
                             lambda B: [K.logsumexp(B, axis=1)],
                             initial_states,
                             U,
                             mask)
    return last_alpha[:, 0]


def _forward(x, reduce_step, initial_states, U, mask=None):
    def _forward_step(energy_matrix_t, states):
        alpha_tm1 = states[-1]
        new_states = reduce_step(K.expand_dims(alpha_tm1, 2) + energy_matrix_t)
        return new_states[0], new_states

    U_shared = K.expand_dims(K.expand_dims(U, 0), 0)

    if mask is not None:
        mask = K.cast(mask, K.floatx())
        mask_U = K.expand_dims(K.expand_dims(mask[:, :-1] * mask[:, 1:], 2), 3)
        U_shared = U_shared * mask_U

    inputs = K.expand_dims(x[:, 1:, :], 2) + U_shared
    inputs = K.concatenate([inputs, K.zeros_like(inputs[:, -1:, :, :])], axis=1)

    last, values, _ = K.rnn(_forward_step, inputs, initial_states)
    return last, values

def batch_gather(reference, indices):
    ref_shape = K.shape(reference)
    batch_size = ref_shape[0]
    n_classes = ref_shape[1]
    flat_indices = K.arange(0, batch_size) * n_classes + K.flatten(indices)
    return K.gather(K.flatten(reference), flat_indices)

def _backward(gamma, mask):
    gamma = K.cast(gamma, 'int32')

    def _backward_step(gamma_t, states):
        y_tm1 = K.squeeze(states[0], 0)
        y_t = batch_gather(gamma_t, y_tm1)
        return y_t, [K.expand_dims(y_t, 0)]

    initial_states = [K.expand_dims(K.zeros_like(gamma[:, 0, 0]), 0)]
    _, y_rev, _ = K.rnn(_backward_step,
                        gamma,
                        initial_states,
                        go_backwards=True)
    y = K.reverse(y_rev, 1)

    if mask is not None:
        mask = K.cast(mask, dtype='int32')
        # mask output
        y *= mask
        # set masked values to -1
        y += -(1 - mask)
    return y

class ChainCRF(Layer):
    def __init__(self, init='glorot_uniform',
                 U_regularizer=None,
                 b_start_regularizer=None,
                 b_end_regularizer=None,
                 U_constraint=None,
                 b_start_constraint=None,
                 b_end_constraint=None,
                 weights=None,
                 **kwargs):
        super(ChainCRF, self).__init__(**kwargs)
        self.init = initializers.get(init)
        self.U_regularizer = regularizers.get(U_regularizer)
        self.b_start_regularizer = regularizers.get(b_start_regularizer)
        self.b_end_regularizer = regularizers.get(b_end_regularizer)
        self.U_constraint = constraints.get(U_constraint)
        self.b_start_constraint = constraints.get(b_start_constraint)
        self.b_end_constraint = constraints.get(b_end_constraint)

        self.initial_weights = weights

        self.supports_masking = True
        self.uses_learning_phase = True
        self.input_spec = [InputSpec(ndim=3)]

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 3
        return (input_shape[0], input_shape[1], input_shape[2])

    def compute_mask(self, input, mask=None):
        if mask is not None:
            return K.any(mask, axis=1)
        return mask

    def _fetch_mask(self):
        mask = None
        if self._inbound_nodes:
            mask = self._inbound_nodes[0].input_masks[0]
        return mask

    def build(self, input_shape):
        assert len(input_shape) == 3
        n_classes = input_shape[2]
        n_steps = input_shape[1]
        assert n_steps is None or n_steps >= 2
        self.input_spec = [InputSpec(dtype=K.floatx(),
                                     shape=(None, n_steps, n_classes))]

        self.U = self.add_weight((n_classes, n_classes),
                                 initializer=self.init,
                                 name='U',
                                 regularizer=self.U_regularizer,
                                 constraint=self.U_constraint)

        self.b_start = self.add_weight((n_classes, ),
                                       initializer='zero',
                                       name='b_start',
                                       regularizer=self.b_start_regularizer,
                                       constraint=self.b_start_constraint)

        self.b_end = self.add_weight((n_classes, ),
                                     initializer='zero',
                                     name='b_end',
                                     regularizer=self.b_end_regularizer,
                                     constraint=self.b_end_constraint)

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

        self.built = True

    def call(self, x, mask=None):
        y_pred = viterbi_decode(x, self.U, self.b_start, self.b_end, mask)
        nb_classes = self.input_spec[0].shape[2]
        y_pred_one_hot = K.one_hot(y_pred, nb_classes)
        return K.in_train_phase(x, y_pred_one_hot)

    def loss(self, y_true, y_pred):
        """Linear Chain Conditional Random Field loss function.
        """
        mask = self._fetch_mask()
        return chain_crf_loss(y_true, y_pred, self.U, self.b_start, self.b_end, mask)

    def sparse_loss(self, y_true, y_pred):
        """Linear Chain Conditional Random Field loss function with sparse
        tag sequences.
        """
        y_true = K.cast(y_true, 'int32')
        y_true = K.squeeze(y_true, 2)
        mask = self._fetch_mask()
        return sparse_chain_crf_loss(y_true, y_pred, self.U, self.b_start, self.b_end, mask)

In [169]:
#from layers import ChainCRF
from keras.optimizers import Nadam
import tensorflow as tf
with tf.device('/gpu:0'):
    word_embeddings = get_embeddings(nlp.vocab)
    words_input = Input(shape=(None,),dtype='int32',name='words_input')
    words = Embedding(input_dim=word_embeddings.shape[0], 
                      output_dim=word_embeddings.shape[1], 
                      weights=[word_embeddings], 
                      trainable=False)(words_input)

    #output of the Embedding layer will be a 3D tensor of shape (samples, sequence_length, embedding_dim).
    character_input=Input(shape=(None,52,),name='char_input')
    embed_char_out=TimeDistributed(Embedding(len(char_indices),100,
                                             embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), #maybe change from 0 to 1
                                             name='char_embedding')(character_input)

    dropout= Dropout(0.5)(embed_char_out)
    conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout) #extra convolutional Network
    maxpool_out=TimeDistributed(MaxPooling1D(52))(conv1d_out)
    char = TimeDistributed(Flatten())(maxpool_out)
    char = Dropout(0.5)(char)
    output = keras.layers.concatenate([words, char]) #concatenation of words and chars to char_embeddings
    context_rep = Bidirectional(LSTM(2*100, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
    #ntime_steps = tf.shape(context_rep)[1]
    #len(ent2id)=9
    output = Dense(100, activation='softmax')(context_rep)
    output = Dense(9)(output)

    #output = TimeDistributed(Dense(9, activation='softmax'))(context_rep)

    #Add Chain CRF Layer here:
    crf=ChainCRF()
    pred=crf(output)

    #Concatenate both inputs together
    #sequence_lengths = Input(batch_shape=(None, 1), dtype='int32')
    model = Model(inputs=[words_input, character_input], outputs=[pred]) #accepts 1 Batch at a time
    model.compile(loss=crf.loss,optimizer=Nadam(lr=0.001),metrics=['accuracy'])
    #model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam(lr=0.001), metrics=['categorical_accuracy'])
    model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 100 8800        char_input[0][0]                 
__________________________________________________________________________________________________
dropout_13 (Dropout)            (None, None, 52, 100 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_19 (TimeDistri (None, None, 52, 30) 9030        dropout_13[0][0]                 
__________________________________________________________________________________________________
time_distr

In [170]:
#Check shapes of array
output.shape

TensorShape([Dimension(None), Dimension(None), Dimension(9)])

In [171]:
pred.shape

TensorShape([Dimension(None), Dimension(None), Dimension(9)])

In [164]:
train_batches = (([i,j],k) for i,j,k in train_batches)

In [172]:
correctLabels=[]
predLabels = []
class Metrics(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        prediction = model.predict([train_word,train_char]).flatten()
        prediction = np.argmax(prediction, -1)
        predLabels.append(prediction)
        correctLabels.append(train_labels.reshape(train_labels.shape[0],train_labels.shape[1]).flatten())
        f1s=compute_f1(predLabels, correctLabels, id2ent) #id2ent=ids to entity 
        return f1s

In [175]:
metrics = Metrics()

In [177]:
#implemeent Callback here
model.fit([train_word, train_char], train_labels,batch_size=100, epochs=1)#,callbacks=[metrics])#callbacks=[metrics]

Epoch 1/1
14041/14041 [==============================] - 111s 8ms/step - loss: 54.3437 - acc: 1.0000


In [178]:
#with F1 Callback
#f1_callback = F1Callback()
model.fit([train_word, train_char], train_labels,batch_size=20, epochs=4) #callbacks=[f1_callback]

Epoch 1/4
14041/14041 [==============================] - 419s 30ms/step - loss: 14.5710 - acc: 1.0000
Epoch 2/4
14041/14041 [==============================] - 419s 30ms/step - loss: 3.6688 - acc: 1.0000
Epoch 3/4
14041/14041 [==============================] - 418s 30ms/step - loss: 1.7202 - acc: 1.0000
Epoch 4/4
14041/14041 [==============================] - 418s 30ms/step - loss: 0.9729 - acc: 1.0000


In [179]:
model.predict([dev_word,dev_char])

array([[[ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.]],

       ..., 
       [[ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0.,  0.],
        [ 1.,  0.,  0., ...,  0.,  0., 

In [180]:
y_p=model.predict([dev_word,dev_char])

In [181]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

val_f1s = []
val_recalls = []
val_precisions = []
clasf_report = []

y_pred = np.argmax(y_p, axis=-1).flatten('C')
y_t = dev_labels
y_true = np.argmax(y_t, axis=-1).flatten('C')
_val_f1 = f1_score(y_true, y_pred, average='weighted')
_val_recall = recall_score(y_true, y_pred, average='weighted')
_val_precision = precision_score(y_true, y_pred, average='weighted')
_clasf_report = classification_report(y_true, y_pred)
val_f1s.append(_val_f1)
val_recalls.append(_val_recall)
val_precisions.append(_val_precision)
clasf_report.append(_clasf_report)
print("\n— val_f1: {} — val_precision: {} — val_recall {}".format(_val_f1, _val_precision, _val_recall))


— val_f1: 1.0 — val_precision: 1.0 — val_recall 1.0


In [ ]:
train_generator = (([i,j],k) for i,j,k in train_batches)
dev_generator = (([i,j],k) for i,j,k in dev_batches)

In [28]:
#Load numpy arrays here:
train_labels = np.load('train_labels.npy')
train_word = np.load('train_word.npy')
train_char = np.load('train_char.npy')

In [182]:
model.train_on_batch([train_word, train_char],train_labels)

ResourceExhaustedError: OOM when allocating tensor with shape[2106150,52,100]
	 [[Node: char_embedding_6/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/gpu:0"](char_embedding_6/embeddings/read, char_embedding_6/Cast)]]
	 [[Node: chain_crf_7/strided_slice_10/_1869 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_5435_chain_crf_7/strided_slice_10", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'char_embedding_6/Gather', defined at:
  File "/home/ubuntu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <m

In [183]:
epochs=100
for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(list(train_batches)))
    for idx, batch in enumerate(train_batches):
        batch_word, batch_char, batch_labels = batch
        model.train_on_batch([batch_word, batch_char], batch_labels)
        a.update(idx)
    print(' ')

Epoch 0/100
 
Epoch 1/100
 
Epoch 2/100
 
Epoch 3/100
 
Epoch 4/100
 
Epoch 5/100
 
Epoch 6/100
 
Epoch 7/100
 
Epoch 8/100
 
Epoch 9/100
 
Epoch 10/100
 
Epoch 11/100
 
Epoch 12/100
 
Epoch 13/100
 
Epoch 14/100
 
Epoch 15/100
 
Epoch 16/100
 
Epoch 17/100
 
Epoch 18/100
 
Epoch 19/100
 
Epoch 20/100
 
Epoch 21/100
 
Epoch 22/100
 
Epoch 23/100
 
Epoch 24/100
 
Epoch 25/100
 
Epoch 26/100
 
Epoch 27/100
 
Epoch 28/100
 
Epoch 29/100
 
Epoch 30/100
 
Epoch 31/100
 
Epoch 32/100
 
Epoch 33/100
 
Epoch 34/100
 
Epoch 35/100
 
Epoch 36/100
 
Epoch 37/100
 
Epoch 38/100
 
Epoch 39/100
 
Epoch 40/100
 
Epoch 41/100
 
Epoch 42/100
 
Epoch 43/100
 
Epoch 44/100
 
Epoch 45/100
 
Epoch 46/100
 
Epoch 47/100
 
Epoch 48/100
 
Epoch 49/100
 
Epoch 50/100
 
Epoch 51/100
 
Epoch 52/100
 
Epoch 53/100
 
Epoch 54/100
 
Epoch 55/100
 
Epoch 56/100
 
Epoch 57/100
 
Epoch 58/100
 
Epoch 59/100
 
Epoch 60/100
 
Epoch 61/100
 
Epoch 62/100
 
Epoch 63/100
 
Epoch 64/100
 
Epoch 65/100
 
Epoch 66/100
 
Epoch

In [184]:
predLabels, correctLabels = tag_dataset(dev_batch)
pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)

NameError: name 'tag_dataset' is not defined

In [41]:
preds.argmax(axis=-1)

array([[5, 5, 6, 6, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]])

In [25]:
LABELS

[u'LOC ', u'PER ', u'MISC ', u'ORG ', u'O ']

In [104]:
preds.shape

(1, 59, 6)

In [94]:
batch_char.shape

(1, 59, 52)

In [ ]:
train_labels = np.argmax(train_labels, axis=2).reshape((len(train_labels), 150, 1))

In [159]:
for i in range(10):
    i_start, i_end = i*200, (i+1)*200
    print(i_start, i_end)
    model.train_on_batch([train_word[i_start:i_end, 0:20], train_char[i_start:i_end, 0:20,:]], train_labels[i_start:i_end, 0:20])

0 200
200 400
400 600
600 800
800 1000
1000 1200
1200 1400
1400 1600
1600 1800
1800 2000


In [164]:
docs = get_ner_sentences('eng.testa')
wx.index = list(docs)

1000 Far East gold traders thumped foreheads in frustration at the market 's foot-dragging this week and forecast on Friday that next week would not be much better . 
2000 Crystal Palace 4 1 2 1 4 3 5 
3000 South Korea in 1993 unconditionally repatriated Li In-mo , a nothern partisan seized by the South during the war and jailed for more than three decades . 


In [171]:
wx.groupby(lambda x: len(x.iloc[0]>0))

AttributeError: 'spacy.tokens.doc.Doc' object has no attribute 'iloc'

In [169]:
wx.iloc[0]

0      168920
1        9860
2       10270
3        3876
4       17661
5       10186
6      217839
7       52637
8           0
9           0
10          0
11          0
12          0
13          0
14          0
15          0
16          0
17          0
18          0
19          0
20          0
21          0
22          0
23          0
24          0
25          0
26          0
27          0
28          0
29          0
        ...  
120         0
121         0
122         0
123         0
124         0
125         0
126         0
127         0
128         0
129         0
130         0
131         0
132         0
133         0
134         0
135         0
136         0
137         0
138         0
139         0
140         0
141         0
142         0
143         0
144         0
145         0
146         0
147         0
148         0
149         0
Name: (CRICKET, -, LEICESTERSHIRE, TAKE, OVER, AT, TOP, AFTER, INNINGS, VICTORY, .), Length: 150, dtype: int32

In [142]:
sent_lens = (word_X>0).sum(axis=1)

In [152]:
train_char[i*100:(i+1)*100].shape

(100, 150, 52)

In [144]:
Counter(sent_lens)

Counter({0: 13,
         1: 55,
         2: 138,
         3: 153,
         4: 399,
         5: 279,
         6: 182,
         7: 153,
         8: 217,
         9: 114,
         10: 86,
         11: 60,
         12: 53,
         13: 57,
         14: 49,
         15: 60,
         16: 53,
         17: 68,
         18: 63,
         19: 58,
         20: 70,
         21: 54,
         22: 52,
         23: 44,
         24: 63,
         25: 55,
         26: 60,
         27: 45,
         28: 48,
         29: 51,
         30: 47,
         31: 42,
         32: 38,
         33: 44,
         34: 40,
         35: 33,
         36: 26,
         37: 19,
         38: 15,
         39: 17,
         40: 21,
         41: 9,
         42: 10,
         43: 5,
         44: 7,
         45: 1,
         46: 4,
         47: 3,
         48: 2,
         49: 2,
         50: 6,
         51: 3,
         53: 1,
         54: 1,
         56: 1,
         59: 1})

In [141]:
for i,g in wx.groupby(lambda x: x>0):
    print(i,g.shape)

False (1, 150)
True (3249, 150)


In [ ]:
batch_size = 100
    batch = []
    for idx, doc in enumerate(docs):
        batch.append(doc)
        if idx%batch_size==(batch_size-1):
            word_X = get_word_emb_features(batch, 150)
            char_X = get_char_emb_features(batch)
            batch=[]
#             print(idx)
            #print((idx-batch_size+1 ,(idx+1)))
            yield word_X, char_X, doc_labels[(idx-batch_size+1) : (idx+1),], LABELS

In [82]:
for words, chars, labels, LABELS in get_batch_data('eng.train'):
    print(words.shape, chars.shape, labels.shape)
    #break
    

(100, 150) (100, 150, 52) (100, 150, 6)
(100, 150) (100, 150, 52) (100, 150, 6)
(100, 150) (100, 150, 52) (100, 150, 6)
(100, 150) (100, 150, 52) (100, 150, 6)
(100, 150) (100, 150, 52) (100, 150, 6)
(100, 150) (100, 150, 52) (100, 150, 6)


KeyboardInterrupt: 

In [93]:
words.shape

(100, 150)

In [87]:
char_indices

{u'\n': 0,
 u' ': 1,
 u'!': 2,
 u'"': 3,
 u'#': 4,
 u'$': 5,
 u'%': 6,
 u'&': 7,
 u"'": 8,
 u'(': 9,
 u')': 10,
 u'*': 11,
 u'+': 12,
 u',': 13,
 u'-': 14,
 u'.': 15,
 u'/': 16,
 u'0': 17,
 u'1': 18,
 u'2': 19,
 u'3': 20,
 u'4': 21,
 u'5': 22,
 u'6': 23,
 u'7': 24,
 u'8': 25,
 u'9': 26,
 u':': 27,
 u';': 28,
 u'=': 29,
 u'?': 30,
 u'@': 31,
 u'A': 32,
 u'B': 33,
 u'C': 34,
 u'D': 35,
 u'E': 36,
 u'F': 37,
 u'G': 38,
 u'H': 39,
 u'I': 40,
 u'J': 41,
 u'K': 42,
 u'L': 43,
 u'M': 44,
 u'N': 45,
 u'O': 46,
 u'P': 47,
 u'Q': 48,
 u'R': 49,
 u'S': 50,
 u'T': 51,
 u'U': 52,
 u'V': 53,
 u'W': 54,
 u'X': 55,
 u'Y': 56,
 u'Z': 57,
 u'[': 58,
 u']': 59,
 u'`': 60,
 u'a': 61,
 u'b': 62,
 u'c': 63,
 u'd': 64,
 u'e': 65,
 u'f': 66,
 u'g': 67,
 u'h': 68,
 u'i': 69,
 u'j': 70,
 u'k': 71,
 u'l': 72,
 u'm': 73,
 u'n': 74,
 u'o': 75,
 u'p': 76,
 u'q': 77,
 u'r': 78,
 u's': 79,
 u't': 80,
 u'u': 81,
 u'v': 82,
 u'w': 83,
 u'x': 84,
 u'y': 85,
 u'z': 86,
 u'|': 87}

In [99]:
words>0

array([[ True,  True, False, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ..., 
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True, False, ..., False, False, False]], dtype=bool)

In [86]:
epochs = 50
for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    
    for i_batch, batch in enumerate(get_batch_data('eng.train')):
        words, chars, labels, LABELS = batch
        #dirty fix
        print('batch:', i_batch)
        print(words.shape, chars.shape, labels.shape)
        #batch_size=100
        labels = np.argmax(labels, axis=2).reshape((100, 150, 1))
        model.train_on_batch([words,  chars], labels)
    
    print(' ')

Epoch 0/50
batch: 0
(100, 150) (100, 150, 52) (100, 150, 6)
batch: 1
(100, 150) (100, 150, 52) (100, 150, 6)
batch: 2
(100, 150) (100, 150, 52) (100, 150, 6)
batch: 3
(100, 150) (100, 150, 52) (100, 150, 6)
batch: 4
(100, 150) (100, 150, 52) (100, 150, 6)
batch: 5
(100, 150) (100, 150, 52) (100, 150, 6)
batch: 6
(100, 150) (100, 150, 52) (100, 150, 6)
batch: 7
(100, 150) (100, 150, 52) (100, 150, 6)
batch: 8
(100, 150) (100, 150, 52) (100, 150, 6)
1000 The youth side replied with 246 for seven . 
batch: 9
(100, 150) (100, 150, 52) (100, 150, 6)
batch: 10
(100, 150) (100, 150, 52) (100, 150, 6)
batch: 11
(100, 150) (100, 150, 52) (100, 150, 6)
batch: 12
(100, 150) (100, 150, 52) (100, 150, 6)
batch: 13
(100, 150) (100, 150, 52) (100, 150, 6)
batch: 14
(100, 150) (100, 150, 52) (100, 150, 6)


KeyboardInterrupt: 

(100, 150)

In [90]:
labels.reshape()

TypeError: reshape() takes exactly 1 argument (0 given)

In [48]:
np.argmax(labels, axis=2).reshape((1000, 150, 1))

array([[[4],
        [5],
        [3],
        ..., 
        [0],
        [0],
        [0]],

       [[2],
        [2],
        [0],
        ..., 
        [0],
        [0],
        [0]],

       [[1],
        [5],
        [0],
        ..., 
        [0],
        [0],
        [0]],

       ..., 
       [[1],
        [5],
        [5],
        ..., 
        [0],
        [0],
        [0]],

       [[2],
        [2],
        [5],
        ..., 
        [0],
        [0],
        [0]],

       [[5],
        [5],
        [5],
        ..., 
        [0],
        [0],
        [0]]])

In [58]:
dev_word_X = get_ner_sentences('eng.train')
dev_word_X = get_word_emb_features(dev_word_X, 150)

dev_char_X = get_ner_sentences('eng.testb')
dev_char_X = get_char_emb_features(dev_char_X)

1000 The youth side replied with 246 for seven . 
2000 Unseeded Patrick Rafter recorded the most noteworthy result as he upset sixth-seeded American MaliVai Washington 6-2 6-1 in just 50 minutes . 
3000 Their release has been negotiated by Islamist writer Ismail Nacar as part of a wider effort , partly backed by Prime Minister Necmettin Erbakan , to find a political solution to Turkey 's Kurdish problem . 
4000 Lille 3 Rennes 1 
5000 Waqar Younis not out 0 
6000 Thai army commanders reject the explanation , saying they have evidence the Burmese army supplies and directs the renegade ethnic minority splinter faction . 
7000 But Stich did n't want to play that game . 
8000 - Fears of an Israeli operation causes the redistribution of Syrian troops locations in Lebanon . 
9000 But Meri , 67 , has been accused in parliament of taking too much power and not always consulting parliamentarians before making decisions . 
10000 Men 's 3,000 metres : 
11000 Austrian composer and organist , he wro

7
2
9
8
8
8
8
8
9
8
8
7
2
8
8
8
8
8
8
8
8
8
9
10
8
4
13
12
6
13
13
13
15
13
13
13
13
13
10
5
4
4
4
4
4
8
4
4
8
8
9
2
10
3
17
18
3
14
3
17
11
9
16
9
10
12
9
9
9
9
9
9
12
7
9
9
9
9
9
9
9
9
9
9
7
7
11
8
8
8
11
8
9
11
8
8
7
8
8
8
8
8
8
8
8
8
8
11
9
31
33
21
26
33
25
8
6
4
8
14
7
13
14
8
3
11
11
13
6
6
5
7
7
10
3
13
15
5
13
15
4
13
8
10
6
5
5
4
4
4
4
15
6
8
8
8
8
8
8
8
8
6
6
5
4
4
4
3
4
5
5
4
3
4
4
4
4
4
5
3
4
7
6
10
22
5
3
5
1
12
4
1
16
4
1
26
4
2
9
6
26
42
26
18
21
16
30
7
6
4
5
16
3
5
3
15
4
4
1
16
4
1
13
4
1
10
4
1
5
1
10
4
1
15
3
4
2
6
6
4
15
8
2
8
8
8
8
8
9
8
8
8
8
2
9
8
8
8
8
9
9
8
8
9
2
8
8
8
8
10
8
8
8
8
8
2
8
9
9
8
8
8
8
10
8
9
6
6
5
15
9
2
8
8
8
9
8
9
8
8
9
8
8
8
8
8
9
8
8
8
8
9
2
8
9
8
9
8
8
8
8
8
8
8
8
8
10
9
8
9
8
8
9
8
8
8
8
2
8
8
8
8
8
8
8
8
8
8
9
9
8
8
8
8
8
8
8
8
8
9
8
8
2
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
9
8
8
8
8
11
6
29
47
25
28
31
8
6
6
5
2
4
4
5
4
4
2
4
6
11
12
2
4
4
2
5
12
4
4
5
14
3
8
6
6
4
2
4
4
4
4
5
4
5
4
2
4
4
4
5
4
4
5
6
5
5
4
7
5
4
4
6
5
4
4
4
4
4
5
5
4
5

In [59]:
train_word_X = get_ner_sentences('eng.train')
train_word_X = get_word_emb_features(train_word_X, 150)

train_char_X = get_ner_sentences('eng.testb')
train_char_X = get_char_emb_features(train_char_X)

1000 The youth side replied with 246 for seven . 
2000 Unseeded Patrick Rafter recorded the most noteworthy result as he upset sixth-seeded American MaliVai Washington 6-2 6-1 in just 50 minutes . 
3000 Their release has been negotiated by Islamist writer Ismail Nacar as part of a wider effort , partly backed by Prime Minister Necmettin Erbakan , to find a political solution to Turkey 's Kurdish problem . 
4000 Lille 3 Rennes 1 
5000 Waqar Younis not out 0 
6000 Thai army commanders reject the explanation , saying they have evidence the Burmese army supplies and directs the renegade ethnic minority splinter faction . 
7000 But Stich did n't want to play that game . 
8000 - Fears of an Israeli operation causes the redistribution of Syrian troops locations in Lebanon . 
9000 But Meri , 67 , has been accused in parliament of taking too much power and not always consulting parliamentarians before making decisions . 
10000 Men 's 3,000 metres : 
11000 Austrian composer and organist , he wro

8
9
10
8
4
13
12
6
13
13
13
15
13
13
13
13
13
10
5
4
4
4
4
4
8
4
4
8
8
9
2
10
3
17
18
3
14
3
17
11
9
16
9
10
12
9
9
9
9
9
9
12
7
9
9
9
9
9
9
9
9
9
9
7
7
11
8
8
8
11
8
9
11
8
8
7
8
8
8
8
8
8
8
8
8
8
11
9
31
33
21
26
33
25
8
6
4
8
14
7
13
14
8
3
11
11
13
6
6
5
7
7
10
3
13
15
5
13
15
4
13
8
10
6
5
5
4
4
4
4
15
6
8
8
8
8
8
8
8
8
6
6
5
4
4
4
3
4
5
5
4
3
4
4
4
4
4
5
3
4
7
6
10
22
5
3
5
1
12
4
1
16
4
1
26
4
2
9
6
26
42
26
18
21
16
30
7
6
4
5
16
3
5
3
15
4
4
1
16
4
1
13
4
1
10
4
1
5
1
10
4
1
15
3
4
2
6
6
4
15
8
2
8
8
8
8
8
9
8
8
8
8
2
9
8
8
8
8
9
9
8
8
9
2
8
8
8
8
10
8
8
8
8
8
2
8
9
9
8
8
8
8
10
8
9
6
6
5
15
9
2
8
8
8
9
8
9
8
8
9
8
8
8
8
8
9
8
8
8
8
9
2
8
9
8
9
8
8
8
8
8
8
8
8
8
10
9
8
9
8
8
9
8
8
8
8
2
8
8
8
8
8
8
8
8
8
8
9
9
8
8
8
8
8
8
8
8
8
9
8
8
2
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
9
8
8
8
8
11
6
29
47
25
28
31
8
6
6
5
2
4
4
5
4
4
2
4
6
11
12
2
4
4
2
5
12
4
4
5
14
3
8
6
6
4
2
4
4
4
4
5
4
5
4
2
4
4
4
5
4
4
5
6
5
5
4
7
5
4
4
6
5
4
4
4
4
4
5
5
4
5
4
5
4
5
4
4
12
6
29
40
34
21
21
10
6
24
2

In [11]:
word_embeddings = get_embeddings(nlp.vocab)

In [37]:
dev_word_X = get_ner_sentences('eng.testb')
dev_word_X = get_word_emb_features(dev_word_X, max_length_sent)

1000 Woman charged over N. Ireland arms find . 
2000 One of the dead was a civilian passer-by . 
3000 Chesterfield 21 11 4 6 22 16 37 


In [17]:
def get_ner_bio(filex):
    sent_count = 0
    sent = []
    with io.open(filex, encoding='utf-8') as fl:
        for line in fl:
            line = line.strip()
            toks = line.split()
            #sentence break
            if len(toks)<1:
                if len(sent)>0:
                    sent_count+=1
                    if sent_count%100==0:
                        print(sent_count, sent)
                    res = sent
                    sent = []                   
                    yield res
                    
            #sentence start
            elif toks[0]==u'-DOCSTART-':
                sent=[]
            else:
                sent.append(toks[3]+' ')

                
def pad_hotencode_labels(train_labels, BIO_LABELS, max_length=150, bio=False):
        
    if bio:
        LABELS = BIO_LABELS
        ent2id = dict(zip(LABELS, 1+np.arange(len(LABELS))))
        train_encids = [[ent2id[i] for i in ents_list] for ents_list in train_labels ]
    else:
        LABELS = list(set([i.split('-')[-1] for i in BIO_LABELS]))
   
        print('Labels >>>>>>>>>>>>>>>>>>>>>>>>>>>', LABELS)

        ent2id_wo_BIO = dict(zip(LABELS, 1+np.arange(len(LABELS))))

        print(ent2id_wo_BIO)
        #train_encids = [[ent2id[i] for i in ents_list] for ents_list in train_labels ]
        train_encids = [[ent2id_wo_BIO[i.split('-')[-1]] for i in ents_list] for ents_list in train_labels ]
            
    encoder = OneHotEncoder()
    encoder.fit(np.arange(len(LABELS)+1).reshape(len(LABELS)+1,1))

    Xs = np.zeros((len(train_labels), max_length), dtype='int32')

    for i in range(len(train_encids)):
        sent=train_encids[i]
        for idx, enc in enumerate(sent):
            #print(idx)
            try:
                Xs[i, idx]=enc
            except:
                print(sent)
                
    tr = Xs.reshape(Xs.shape[0], Xs.shape[1], 1)
    tr_he = np.array([encoder.transform(tr[i,:,:]).toarray() for i in range(tr.shape[0])])
    return tr_he

train_labels = list(get_ner_bio('eng.train'))
BIO_LABELS = list(set([i for sub_list in train_labels for i in sub_list]))
print(BIO_LABELS)
max_length_sent = 150

LABELS = list(set([i.split('-')[-1] for i in BIO_LABELS]))
train_labels = pad_hotencode_labels(train_labels, BIO_LABELS, max_length=max_length_sent)

dev_labels = list(get_ner_bio('eng.testb'))
dev_labels = pad_hotencode_labels(dev_labels, BIO_LABELS,max_length=max_length_sent)

100 [u'O ', u'I-MISC ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'I-LOC ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ']
200 [u'I-ORG ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ']
300 [u'O ', u'I-ORG ', u'I-ORG ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'I-MISC ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ']
400 [u'I-PER ', u'I-PER ', u'O ', u'I-LOC ', u'I-LOC ', u'O ', u'O ', u'O ', u'O ', u'I-PER ', u'I-PER ', u'I-PER ']
500 [u'I-PER ', u'I-PER ', u'O ', u'I-LOC ', u'I-LOC ', u'O ', u'O ', u'O ', u'O ', u'I-PER ', u'I-PER ']
600 [u'I-ORG ', u'I-ORG ', u'O ', u'O ', u'O ', u'O ', u'O ']
700 [u'I-PER ', u'I-PER ', u'O ', u'I-LOC ', u'O ', u'O ', u'I-PER ', u'I-PER ', u'O ', u'I-LOC ', u'O ']
800 [u'I-ORG ', u'I-ORG ', u'O ', u'O ', u'O ', u'O ']
900 [u'I-

9600 [u'O ', u'I-ORG ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'I-ORG ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ']
9700 [u'I-LOC ', u'O ']
9800 [u'I-LOC ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'I-LOC ', u'O ']
9900 [u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'I-PER ', u'O ']
10000 [u'O ', u'O ', u'O ', u'O ', u'O ']
10100 [u'O ', u'I-LOC ', u'O ', u'O ']
10200 [u'I-LOC ', u'O ']
10300 [u'I-ORG ', u'O ', u'I-LOC ', u'O ', u'O ', u'I-ORG ', u'O ', u'I-LOC ', u'O ', u'O ', u'I-ORG ', u'O ', u'I-LOC ', u'O ', u'O ', u'I-ORG ', u'O ', u'I-LOC ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ', u'O ']
10400 [u'I-PER ', u'I-PER ', u'O ', u'I-LOC ', u'I-LOC ', u'O ', u'O ', u'I-PER ', u'I-PER ', u'I-PER ', u'O ', u'I-LOC ', u'I-LOC ', u'O ', u'O ', u'O ']
10500 [u'I-ORG ', u'O ', u'I-ORG ', u'O ']
10600 [u'O ', u'O ', u'O ', u'O ', u'O ']
10700 [